# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [3]:
%%time
ratings_us1 = ratings[ratings['userId'] == 1]
ratings_us = ratings_us1[['rating']]
def some_dict(some_df):
    data_lst = {}
    for _,__ in ratings_us.items():
        for ___ in __:
            if ___ not in data_lst:
                data_lst[___] = 1
            else:
                data_lst[___] += 1
    return data_lst

def show(some_dict):
    for _, __ in some_dict.items():
        print(f"Пользователь ID-1: Оценка:{_}   Колличество:{__}")

show(some_dict(ratings_us))
# print(ratings_us1, len(ratings_us1))

Пользователь ID-1: Оценка:2.5   Колличество:3
Пользователь ID-1: Оценка:3.0   Колличество:4
Пользователь ID-1: Оценка:2.0   Колличество:7
Пользователь ID-1: Оценка:4.0   Колличество:3
Пользователь ID-1: Оценка:3.5   Колличество:1
Пользователь ID-1: Оценка:1.0   Колличество:2
CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time
firast_id_rait = ratings[ratings.userId == 1].rating.value_counts(ascending=False)
firast_id_rait


CPU times: total: 0 ns
Wall time: 0 ns


2.0    7
3.0    4
2.5    3
4.0    3
1.0    2
3.5    1
Name: rating, dtype: int64

Как это сделать для всех пользователей сразу:

In [5]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5
...,...,...,...,...,...,...,...,...,...,...
667,0,0,0,4,0,28,0,24,0,12
668,0,2,0,0,0,7,0,3,0,8
669,0,0,0,7,0,14,0,12,0,4


In [6]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = '___',
                    margins = True)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,___,2.0,___,7.0,3.0,4.0,1.0,3.0,___,___,20
2,___,2.0,___,4.0,___,36.0,___,23.0,___,11.0,76
3,___,___,___,1.0,3.0,18.0,9.0,11.0,4.0,5.0,51
4,___,5.0,___,5.0,___,23.0,___,52.0,___,119.0,204
5,___,___,1.0,___,3.0,3.0,27.0,42.0,19.0,5.0,100
...,...,...,...,...,...,...,...,...,...,...,...
668,___,2.0,___,___,___,7.0,___,3.0,___,8.0,20
669,___,___,___,7.0,___,14.0,___,12.0,___,4.0,37
670,___,2.0,___,3.0,___,5.0,___,10.0,___,11.0,31


### Упражнение
Какой пользователь выставил больше всех пятерок?

In [7]:
ratings_upd = ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, margins = True)
print(ratings_upd.sort_values(5.0, ascending=False).head(10))

rating   0.5   1.0   1.5   2.0   2.5    3.0    3.5    4.0   4.5    5.0     All
userId                                                                        
All     1101  3326  1687  7271  4449  20064  10538  28750  7723  15095  100004
564        0   152     0   187     0    414      0    707     0    408    1868
232        0    35     0    32     0     96      0    276     0    243     682
242        0     0     0     4     0     25      0    151     0    219     399
547       53    79    58   204   177    411    378    591   226    214    2391
30         0    20     0    86     0    201      2    505     1    196    1011
472        0    23     1    56     5    188     13    315    40    189     830
102        0     8     0    40     0     87      0    369     0    174     678
358        0   114     0    91     0    118      0    156     0    138     617
388        0    27     1    78    10    165     35    297    44    135     792


### Фильтры и вычисления с помощью loc и iloc

In [33]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [36]:
len(log['url'].value_counts())

38

In [49]:
%%time
log_pre = log['url'].value_counts().head()
print(log_pre)
for _, __ in log_pre.items():
    print(_[16:])

https://host.ru/3c19b4ef7371864fa3    541
https://host.ru/108ce4b365afb7b88e    534
https://host.ru/23c64743ad842b9311    532
https://host.ru/1afc5618562365020a    529
https://host.ru/2fcb223266880ccf57    526
Name: url, dtype: int64
3c19b4ef7371864fa3
108ce4b365afb7b88e
23c64743ad842b9311
1afc5618562365020a
2fcb223266880ccf57
CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 2.78 ms


In [56]:
%%time
log['address'] = log_pre['url'].apply(lambda url: url[16:])
log

KeyError: 'url'

In [ ]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

In [ ]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [ ]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

In [ ]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

In [ ]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

Можно и через loc

In [ ]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

Для замера времени сделаем эти операции для набор жанров

In [ ]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

In [ ]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

In [ ]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [ ]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [ ]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [ ]:
from pymystem3 import Mystem

In [ ]:
search = 'курс гривны к рублю рубли рублях'

In [ ]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

In [ ]:
' '.join(lemmas)

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [ ]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})